In [1]:
import sqlite3
import pandas as pd
import numpy as np
from sql_metadata import Parser

In [2]:
from datetime import datetime, timedelta

In [3]:
# This allows multiple outputs from a single jupyter notebook cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
import mplfinance as mpf

In [34]:
FLAG_DEBUG = False # True # 
NUM_DAYS_QUOTE, NUM_DAYS_PLOT = 390, 250
RSI_PERIOD, RSI_AVG, RSI_BAND_WIDTH = 100, 25, 0.6
EMA_FAST, EMA_SLOW, EMA_LONG = 15, 50, 150
EMA_FAST_SCALE = 1.4  # EMA10 band half-width factor
EMA_SLOW_SCALE = 2.0 
MA_VOL = 20
SPAN, OVERLAP = 200, 15
PANID_PRICE, PANID_VOL, PANID_RSI, PANID_SIGNAL = 0, 3, 2, 1
PANEL_RATIOS = (8, 1, 8, 1)
FIGURE_WIDTH, FIGURE_HEIGHT =  17, 13
YELLOW = '#F5D928'
LIGHT_BLACK = '#8F8E83'

FIRST_TIME = True

In [6]:
def _load_stooq_data(file_path):
    """Load historical quotes from http://stooq.com/db/h
    and parse it to Pandas dataframe
    """
    df = pd.read_csv(file_path,index_col=2,parse_dates=False)
    df.reset_index(inplace=True)
    df.drop(columns=['<TICKER>', '<PER>', '<TIME>','<OPENINT>'], axis=1, inplace=True)
    # Avoid reserved words like date, open via suffix '_' for original quotes
    df.rename(columns={'<DATE>' : "date_", '<OPEN>': "open_", '<HIGH>':"high_", 
                   '<LOW>':"low_", '<CLOSE>':"close_", '<VOL>':"volume_"} ,inplace=True)
    df.set_index("date_", inplace=True)
    return df


def _ta_RSI(df, n=14):
    """Calculate Technical Analysis indicator RSI, 0-centered
    """
    # https://github.com/wgong/mplfinance/blob/master/examples/rsi.py
    diff = df.w_p.diff().values
    gains = diff
    losses = -diff
    with np.errstate(invalid='ignore'):
        gains[(gains<0)|np.isnan(gains)] = 0.0
        losses[(losses<=0)|np.isnan(losses)] = 1e-10 # we don't want divide by zero/NaN
    m = (n-1) / n
    ni = 1 / n
    g = gains[n] = np.nanmean(gains[:n])
    l = losses[n] = np.nanmean(losses[:n])
    gains[:n] = losses[:n] = np.nan
    for i,v in enumerate(gains[n:],n):
        g = gains[i] = ni*v + m*g
    for i,v in enumerate(losses[n:],n):
        l = losses[i] = ni*v + m*l
    rs = gains / losses
    return 50 - (100/(1+rs))



def _calculate_ta(df,rsi_period=RSI_PERIOD, avg_period=RSI_AVG, band_width=RSI_BAND_WIDTH):
    """Calculate Technical Analysis indicators such as 
        - EMA (fast,slow,long), EMA-band, 
        - RSI, RSI-band,
        - Vol-avg
    """
    
    df["w_p"] = 0.25*(2*df["close_"] + df["high_"] + df["low_"])
    df["ema_fast"] = df.w_p.ewm(span=EMA_FAST).mean()
    df["ema_slow"] = df.w_p.ewm(span=EMA_SLOW).mean()
    df["ema_long"] = df.w_p.ewm(span=EMA_LONG).mean()

    # range
    hl_mean_fast = (df.high_ - df.low_).ewm(span=int(EMA_FAST/2)).mean()
    df["ema_fast_u"] =  df.ema_fast + 0.5*hl_mean_fast * EMA_FAST_SCALE
    df["ema_fast_d"] =  df.ema_fast - 0.5*hl_mean_fast * EMA_FAST_SCALE

    hl_mean_slow = (df.high_ - df.low_).ewm(span=int(EMA_SLOW/2)).mean()
    df["ema_slow_u"] =  df.ema_slow + 0.5*hl_mean_slow * EMA_SLOW_SCALE
    df["ema_slow_d"] =  df.ema_slow - 0.5*hl_mean_slow * EMA_SLOW_SCALE

    # trim volume to avoid exponential form
    df['vol'] = df['volume_'] / 1000000.0
    df["vol_avg"] = df.vol.ewm(span=MA_VOL).mean()

    df["rsi"] = _ta_RSI(df,n=rsi_period)
    df["rsi_avg"] = df.rsi.ewm(span=avg_period).mean()
    df["rsi_u"] = df["rsi_avg"] + band_width 
    df["rsi_d"] = df['rsi_avg'] - band_width 
    df["rsi_signal"] = df["rsi"] - df["rsi_avg"]

    df["del_ema_slow"] = df.ema_slow.diff().values
    df["del_ema_long"] = df.ema_long.diff().values
    df["del_rsi_avg"] = df.rsi_avg.diff().values

    return df

def _display_df(df, nrows=3):
    print(df.head(nrows))
    print("\n",3*"..","\n")
    print(df.tail(nrows))

In [7]:
create_table_sql = '''CREATE TABLE quote_ta
    (ticker text
    
    , date_ text
	, open_ real
	, high_ real
	, low_ real
	, close_ real
	, volume_ INTEGER
    
	, w_p real
	, ema_fast real
	, ema_slow real
	, ema_long real
	, ema_fast_u real
	, ema_fast_d real
	, ema_slow_u real
	, ema_slow_d real
	, vol real
	, vol_avg real
	, rsi real
	, rsi_avg real
	, rsi_u real
	, rsi_d real
	, rsi_signal real
	, del_ema_slow real
	, del_ema_long real
	, del_rsi_avg real
    )
'''

# https://www.tutorialspoint.com/sqlite/sqlite_indexes.htm
create_index_sql = '''
Create unique index if not exists
ticker_dt on quote_ta ( ticker , date_ )
'''

In [8]:
parser = Parser(create_table_sql)
table_name = parser.tables[0]
table_columns = parser.columns   # open is a key-word, skipped
print(f"table_name: {table_name} \n\ncolumns: {table_columns}")

table_name: quote_ta 

columns: ['ticker', 'date_', 'open_', 'high_', 'low_', 'close_', 'volume_', 'w_p', 'ema_fast', 'ema_slow', 'ema_long', 'ema_fast_u', 'ema_fast_d', 'ema_slow_u', 'ema_slow_d', 'vol', 'vol_avg', 'rsi', 'rsi_avg', 'rsi_u', 'rsi_d', 'rsi_signal', 'del_ema_slow', 'del_ema_long', 'del_rsi_avg']


In [9]:
db_name = "NQ100"
file_tickers = "wl_index-QQQ_NASDAQ100.csv"
file_dict = "ticker-catalog.csv"
file_db = f"db_{db_name}_TEST.sqlite"
FILE_DB = file_db
CHART_TABLE_NAME = "quote_ta"

In [10]:
df_ticker = pd.read_csv(file_tickers)
df_meta = pd.read_csv(file_dict)

In [11]:
_display_df(df_ticker)

  Ticker               Company
0    QQQ                   ETF
1   ATVI   Activision Blizzard
2   ADBE            Adobe Inc.

 ...... 

    Ticker                     Company
99     XEL                 Xcel Energy
100   XLNX                      Xilinx
101     ZM   Zoom Video Communications


In [12]:
_display_df(df_meta)

  ticker  type   exchg                              file_path
0   AADR  etfs  nasdaq  data\daily\us\nasdaq etfs\aadr.us.txt
1   AAXJ  etfs  nasdaq  data\daily\us\nasdaq etfs\aaxj.us.txt
2   ACWI  etfs  nasdaq  data\daily\us\nasdaq etfs\acwi.us.txt

 ...... 

       ticker    type    exchg                                   file_path
12245  YCBD_A  stocks  nysemkt  data\daily\us\nysemkt stocks\ycbd_a.us.txt
12246    ZDGE  stocks  nysemkt    data\daily\us\nysemkt stocks\zdge.us.txt
12247     ZOM  stocks  nysemkt     data\daily\us\nysemkt stocks\zom.us.txt


In [13]:
tickers_all = df_meta['ticker'].to_list()

In [14]:
len(tickers_all), tickers_all[:5]

(12248, ['AADR', 'AAXJ', 'ACWI', 'ACWX', 'ADRE'])

test this ticker

In [15]:
ticker = "qqq".upper()

In [16]:
dd = df_meta[df_meta['ticker'] == ticker].to_dict('records')[0]
dd

{'ticker': 'QQQ',
 'type': 'etfs',
 'exchg': 'nasdaq',
 'file_path': 'data\\daily\\us\\nasdaq etfs\\qqq.us.txt'}

In [17]:
df1 = pd.read_csv(dd['file_path'])

In [18]:
_display_df(df1)

  <TICKER> <PER>    <DATE>  <TIME>  <OPEN>  <HIGH>   <LOW>  <CLOSE>     <VOL>  \
0   QQQ.US     D  19990310       0  44.334  44.361  43.605   44.279  12066145   
1   QQQ.US     D  19990311       0  44.601  44.856  43.624   44.488  22347406   
2   QQQ.US     D  19990312       0  44.333  44.360  43.061   43.414  20164977   

   <OPENINT>  
0          0  
1          0  
2          0  

 ...... 

     <TICKER> <PER>    <DATE>  <TIME>  <OPEN>  <HIGH>   <LOW>  <CLOSE>  \
5805   QQQ.US     D  20220404       0  362.71  369.31  362.44   369.30   
5806   QQQ.US     D  20220405       0  367.81  368.54  359.93   361.10   
5807   QQQ.US     D  20220406       0  355.65  356.78  350.56   353.24   

         <VOL>  <OPENINT>  
5805  45950171          0  
5806  62738661          0  
5807  89720554          0  


In [19]:
# load quotes
df = _load_stooq_data(dd['file_path'])

In [20]:
_display_df(df)

           open_   high_    low_  close_   volume_
date_                                             
19990310  44.334  44.361  43.605  44.279  12066145
19990311  44.601  44.856  43.624  44.488  22347406
19990312  44.333  44.360  43.061  43.414  20164977

 ...... 

           open_   high_    low_  close_   volume_
date_                                             
20220404  362.71  369.31  362.44  369.30  45950171
20220405  367.81  368.54  359.93  361.10  62738661
20220406  355.65  356.78  350.56  353.24  89720554


In [21]:
df = _calculate_ta(df)

In [22]:
_display_df(df)

           open_   high_    low_  close_   volume_       w_p   ema_fast  \
date_                                                                     
19990310  44.334  44.361  43.605  44.279  12066145  44.13100  44.131000   
19990311  44.601  44.856  43.624  44.488  22347406  44.36400  44.255267   
19990312  44.333  44.360  43.061  43.414  20164977  43.56225  43.992822   

           ema_slow   ema_long  ema_fast_u  ...        vol    vol_avg  rsi  \
date_                                       ...                              
19990310  44.131000  44.131000   44.660200  ...  12.066145  12.066145  NaN   
19990311  44.249830  44.248277   44.974867  ...  22.347406  17.463807  NaN   
19990312  44.011409  44.016545   44.794455  ...  20.164977  18.455660  NaN   

          rsi_avg  rsi_u  rsi_d  rsi_signal  del_ema_slow  del_ema_long  \
date_                                                                     
19990310      NaN    NaN    NaN         NaN           NaN           NaN   
19990311

In [23]:
df.reset_index(inplace=True)

In [24]:
df.columns

Index(['date_', 'open_', 'high_', 'low_', 'close_', 'volume_', 'w_p',
       'ema_fast', 'ema_slow', 'ema_long', 'ema_fast_u', 'ema_fast_d',
       'ema_slow_u', 'ema_slow_d', 'vol', 'vol_avg', 'rsi', 'rsi_avg', 'rsi_u',
       'rsi_d', 'rsi_signal', 'del_ema_slow', 'del_ema_long', 'del_rsi_avg'],
      dtype='object')

In [25]:
# add ticker column
df['ticker'] = ticker

In [26]:
_display_df(df)

      date_   open_   high_    low_  close_   volume_       w_p   ema_fast  \
0  19990310  44.334  44.361  43.605  44.279  12066145  44.13100  44.131000   
1  19990311  44.601  44.856  43.624  44.488  22347406  44.36400  44.255267   
2  19990312  44.333  44.360  43.061  43.414  20164977  43.56225  43.992822   

    ema_slow   ema_long  ...    vol_avg  rsi  rsi_avg  rsi_u  rsi_d  \
0  44.131000  44.131000  ...  12.066145  NaN      NaN    NaN    NaN   
1  44.249830  44.248277  ...  17.463807  NaN      NaN    NaN    NaN   
2  44.011409  44.016545  ...  18.455660  NaN      NaN    NaN    NaN   

   rsi_signal  del_ema_slow  del_ema_long  del_rsi_avg  ticker  
0         NaN           NaN           NaN          NaN     QQQ  
1         NaN      0.118830      0.117277          NaN     QQQ  
2         NaN     -0.238421     -0.231731          NaN     QQQ  

[3 rows x 25 columns]

 ...... 

         date_   open_   high_    low_  close_   volume_       w_p  \
5805  20220404  362.71  369.31  362.44

In [27]:
len(table_columns), table_columns

(25,
 ['ticker',
  'date_',
  'open_',
  'high_',
  'low_',
  'close_',
  'volume_',
  'w_p',
  'ema_fast',
  'ema_slow',
  'ema_long',
  'ema_fast_u',
  'ema_fast_d',
  'ema_slow_u',
  'ema_slow_d',
  'vol',
  'vol_avg',
  'rsi',
  'rsi_avg',
  'rsi_u',
  'rsi_d',
  'rsi_signal',
  'del_ema_slow',
  'del_ema_long',
  'del_rsi_avg'])

In [28]:
df = df[table_columns]

In [29]:
_display_df(df)

  ticker     date_   open_   high_    low_  close_   volume_       w_p  \
0    QQQ  19990310  44.334  44.361  43.605  44.279  12066145  44.13100   
1    QQQ  19990311  44.601  44.856  43.624  44.488  22347406  44.36400   
2    QQQ  19990312  44.333  44.360  43.061  43.414  20164977  43.56225   

    ema_fast   ema_slow  ...        vol    vol_avg  rsi  rsi_avg  rsi_u  \
0  44.131000  44.131000  ...  12.066145  12.066145  NaN      NaN    NaN   
1  44.255267  44.249830  ...  22.347406  17.463807  NaN      NaN    NaN   
2  43.992822  44.011409  ...  20.164977  18.455660  NaN      NaN    NaN   

   rsi_d  rsi_signal  del_ema_slow  del_ema_long  del_rsi_avg  
0    NaN         NaN           NaN           NaN          NaN  
1    NaN         NaN      0.118830      0.117277          NaN  
2    NaN         NaN     -0.238421     -0.231731          NaN  

[3 rows x 25 columns]

 ...... 

     ticker     date_   open_   high_    low_  close_   volume_       w_p  \
5805    QQQ  20220404  362.71  369.

In [30]:
numeric_cols = ['w_p', 'ema_fast', 'ema_slow', 'ema_long', 'ema_fast_u', 'ema_fast_d', 'ema_slow_u', 'ema_slow_d',
     'vol', 'vol_avg', 'rsi', 'rsi_avg', 'rsi_u', 'rsi_d', 'rsi_signal', 'del_ema_slow', 'del_ema_long', 'del_rsi_avg']

In [31]:
df.shape, df.columns

((5808, 25),
 Index(['ticker', 'date_', 'open_', 'high_', 'low_', 'close_', 'volume_', 'w_p',
        'ema_fast', 'ema_slow', 'ema_long', 'ema_fast_u', 'ema_fast_d',
        'ema_slow_u', 'ema_slow_d', 'vol', 'vol_avg', 'rsi', 'rsi_avg', 'rsi_u',
        'rsi_d', 'rsi_signal', 'del_ema_slow', 'del_ema_long', 'del_rsi_avg'],
       dtype='object'))

In [32]:
df[numeric_cols] = np.round(df[numeric_cols], 4)

In [33]:
_display_df(df, nrows=2)

  ticker     date_   open_   high_    low_  close_   volume_     w_p  \
0    QQQ  19990310  44.334  44.361  43.605  44.279  12066145  44.131   
1    QQQ  19990311  44.601  44.856  43.624  44.488  22347406  44.364   

   ema_fast  ema_slow  ...      vol  vol_avg  rsi  rsi_avg  rsi_u  rsi_d  \
0   44.1310   44.1310  ...  12.0661  12.0661  NaN      NaN    NaN    NaN   
1   44.2553   44.2498  ...  22.3474  17.4638  NaN      NaN    NaN    NaN   

   rsi_signal  del_ema_slow  del_ema_long  del_rsi_avg  
0         NaN           NaN           NaN          NaN  
1         NaN        0.1188        0.1173          NaN  

[2 rows x 25 columns]

 ...... 

     ticker     date_   open_   high_    low_  close_   volume_       w_p  \
5806    QQQ  20220405  367.81  368.54  359.93  361.10  62738661  362.6675   
5807    QQQ  20220406  355.65  356.78  350.56  353.24  89720554  353.4550   

      ema_fast  ema_slow  ...      vol  vol_avg     rsi  rsi_avg   rsi_u  \
5806  357.5173  354.7759  ...  62.7387  6

#### Create table for the first time

In [35]:
if FIRST_TIME:
    conn = sqlite3.connect(file_db)
    cur = conn.cursor()

    # Create table
    cur.execute(create_table_sql)

    conn.commit()
    conn.close()

load csv to sqlite db
- https://stackoverflow.com/questions/2887878/importing-a-csv-file-into-a-sqlite3-database-table-using-python

In [36]:
conn = sqlite3.connect(file_db)
cur = conn.cursor()

In [37]:
df.to_sql(table_name, conn, if_exists='append', index=False)

#### create index after insert

In [38]:
if FIRST_TIME:
    conn = sqlite3.connect(file_db)
    cur = conn.cursor()

    # Create table
    cur.execute(create_index_sql)

    conn.commit()
    conn.close()

In [39]:
conn = sqlite3.connect(file_db)
cur = conn.cursor()

In [40]:
cur.execute(f'SELECT count(*) FROM {table_name};')
print(cur.fetchone())

(5808,)


In [41]:
sql_stmt = f'''
SELECT distinct ticker 
FROM {table_name}
order by ticker
;
'''
df1 = pd.read_sql(sql_stmt, conn)
df1

,ticker
0,QQQ


In [42]:
FILE_DB = f"C:\\gwgllc\\stooq.com\\db_{db_name}_TEST.sqlite"

In [43]:
def _query_tickers():
    conn = sqlite3.connect(FILE_DB)
    sql_stmt = f'''
    SELECT distinct ticker 
    FROM {table_name}
    order by ticker
    ;
    '''
    df1 = pd.read_sql(sql_stmt, conn)
    conn.close()
    return df1['ticker'].to_list()

def _reformat_df_from_db(df):
    """rename columns for charting, parse date, set_index
    
        pd.to_datetime('20220406')
        # Timestamp('2022-04-06 00:00:00')    
    """
    df.rename(columns={'date_' : "Date", 'open_': "Open", 'high_':"High", 
                   'low_':"Low", 'close_':"Close", 'vol':"Volume"}, inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index("Date", inplace=True)
    return df

def _query_chart_data(tickers, date_range):
    tickers_str = str(tickers).replace('[','(').replace(']',')')
    conn = sqlite3.connect(FILE_DB)
    sql_stmt = f'''
    SELECT * 
    FROM {table_name} 
    where ticker in {tickers_str} 
        and date_ between '{date_range[0]}' and '{date_range[1]}'
    order by ticker, date_
    '''
    df = pd.read_sql(sql_stmt, conn)
    conn.close()
    return _reformat_df_from_db(df)

def _query_date_range():
    conn = sqlite3.connect(FILE_DB)
    sql_stmt = f'''
    SELECT min(date_) as begin_date, max(date_) as end_date
    FROM {CHART_TABLE_NAME};
    '''
    df1 = pd.read_sql(sql_stmt, conn)
    return df1.iloc[0][["begin_date", "end_date"]].to_list()

def _make_date_ranges(range_size=NUM_DAYS_PLOT, range_overlap=10):
    """generate date_range windows
    """
    date_range = _query_date_range()
    min_date = pd.to_datetime(date_range[0])
    max_date = pd.to_datetime(date_range[1])
    date_ranges = []
    for i in range(0, (max_date - min_date).days, range_size):
        date_stop = max_date - timedelta(days=i)
        date_start = date_stop - timedelta(days=range_size+range_overlap)
        date_ranges.append((datetime.strftime(date_start,'%Y-%m-%d'), \
                            datetime.strftime(date_stop,'%Y-%m-%d')))
    return date_ranges

In [39]:
list_date_ranges = [f"{dr[0]} _ {dr[1]}" for dr in _make_date_ranges()]
# list_date_ranges

In [40]:
list_date_ranges.insert(0,' _ ')
list_date_ranges

[' _ ',
 '2021-07-20 _ 2022-04-06',
 '2020-11-12 _ 2021-07-30',
 '2020-03-07 _ 2020-11-22',
 '2019-07-01 _ 2020-03-17',
 '2018-10-24 _ 2019-07-11',
 '2018-02-16 _ 2018-11-03',
 '2017-06-11 _ 2018-02-26',
 '2016-10-04 _ 2017-06-21',
 '2016-01-28 _ 2016-10-14',
 '2015-05-23 _ 2016-02-07',
 '2014-09-15 _ 2015-06-02',
 '2014-01-08 _ 2014-09-25',
 '2013-05-03 _ 2014-01-18',
 '2012-08-26 _ 2013-05-13',
 '2011-12-20 _ 2012-09-05',
 '2011-04-14 _ 2011-12-30',
 '2010-08-07 _ 2011-04-24',
 '2009-11-30 _ 2010-08-17',
 '2009-03-25 _ 2009-12-10',
 '2008-07-18 _ 2009-04-04',
 '2007-11-11 _ 2008-07-28',
 '2007-03-06 _ 2007-11-21',
 '2006-06-29 _ 2007-03-16',
 '2005-10-22 _ 2006-07-09',
 '2005-02-14 _ 2005-11-01',
 '2004-06-09 _ 2005-02-24',
 '2003-10-03 _ 2004-06-19',
 '2003-01-26 _ 2003-10-13',
 '2002-05-21 _ 2003-02-05',
 '2001-09-13 _ 2002-05-31',
 '2001-01-06 _ 2001-09-23',
 '2000-05-01 _ 2001-01-16',
 '1999-08-25 _ 2000-05-11',
 '1998-12-18 _ 1999-09-04']

In [33]:
pd.to_datetime('1998-12-18')

Timestamp('1998-12-18 00:00:00')

In [44]:
def _title_xy(ticker):
    # position title manually
    return {"title": f"{ticker.upper()}",  "x": 0.75, "y": 0.95}

def _color_rsi_avg(v):
    if v > 0: return '#DCF7E5'
    elif v < 0: return '#F6D5F7'
    return YELLOW

def _color_signal(v):
    if v > 0: return 'g'
    elif v < 0: return 'r'
    return YELLOW

def gen_chart(ticker, df, num=0, panid_price=PANID_PRICE, panid_vol=PANID_VOL,
              panid_rsi=PANID_RSI, panid_signal=PANID_SIGNAL, image_path="images"):
    """
    Generate chart using Mplfinance
    """
    ema_fast_u_plot = mpf.make_addplot(df["ema_fast_u"], panel=panid_price, color=LIGHT_BLACK, linestyle="solid")
    ema_fast_d_plot = mpf.make_addplot(df["ema_fast_d"], panel=panid_price, color=LIGHT_BLACK, linestyle="solid")
    ema_slow_plot = mpf.make_addplot(df["ema_slow"], panel=panid_price, color='b', width=2, linestyle="solid")
    ema_long_plot = mpf.make_addplot(df["ema_long"], panel=panid_price, width=2, color='k')  # magenta
    
    rsi_min, rsi_max = np.nanmin(df["rsi"]), np.nanmax(df["rsi"])
    rsi_avg_plot = mpf.make_addplot(df["rsi_avg"], panel=panid_rsi, type="bar", 
                            color=[_color_rsi_avg(v)  for v in df["rsi_avg"]], 
                            ylim=(rsi_min,rsi_max))
    rsi_avg_plot2 = mpf.make_addplot(df["rsi_avg"], panel=panid_rsi, color="b", width=1, linestyle="solid",
                            ylim=(rsi_min,rsi_max))
    rsi_plot = mpf.make_addplot(df["rsi"], panel=panid_rsi, color='r', width=1, #title=f"{ticker}-RSI",
                            fill_between=dict(y1=df["rsi_d"].values,y2=df["rsi_u"].values,alpha=0.15,color='b'),
                            ylim=(rsi_min,rsi_max))
    signal_plot = mpf.make_addplot(df["rsi_signal"], panel=panid_signal, type="bar", 
                            color=[_color_signal(v) for v in df["rsi_signal"]], 
                            ylim=(-1,1))                            

    vol_avg_plot = mpf.make_addplot(df["vol_avg"], panel=panid_vol, color='k')

    plots = [
            # panel-0
            ema_fast_u_plot, ema_fast_d_plot, ema_slow_plot, ema_long_plot # ema_slow_u_plot, ema_slow_d_plot, 
            # panel-1
            , rsi_avg_plot, rsi_plot, rsi_avg_plot2  # , rsi_u_plot, rsi_d_plot 
            # panel-2
            , vol_avg_plot
            # panel-3
            , signal_plot
        ]
    file_img = f"{image_path}/{ticker}-{str(num).zfill(2)}.png"
    print(file_img)
    mpf.plot(df, type='candle', 
            style='yahoo', 
            fill_between=dict(y1=df["ema_fast_d"].values,y2=df["ema_fast_u"].values,alpha=0.15,color='b'),
            panel_ratios=PANEL_RATIOS,
            addplot=plots, 
            title=_title_xy(ticker),
            volume=True, volume_panel=panid_vol, 
            ylabel="", ylabel_lower='',
            xrotation=0,
            datetime_format='%Y-%m-%d',
            savefig=file_img,
            figsize=(FIGURE_WIDTH,FIGURE_HEIGHT),
            tight_layout=True,
            show_nontrading=True
        )  
    return {"ticker": ticker, "file_img": file_img}

In [45]:
df = _query_chart_data(_query_tickers(), ('20210101','20220601'))

In [46]:
df

,ticker,Open,High,Low,Close,volume_,w_p,ema_fast,ema_slow,ema_long,...,Volume,vol_avg,rsi,rsi_avg,rsi_u,rsi_d,rsi_signal,del_ema_slow,del_ema_long,del_rsi_avg
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,QQQ,313.61,313.79,303.72,307.83,45523231,308.2925,307.4427,297.2056,274.3720,...,45.5232,27.8293,8.4733,8.3747,8.9747,7.7747,0.0986,0.4525,0.4553,0.0082
2021-01-05,QQQ,306.82,310.65,306.82,310.37,29464073,309.5525,307.7064,297.6898,274.8380,...,29.4641,27.9850,8.6774,8.3980,8.9980,7.7980,0.2794,0.4842,0.4660,0.0233
2021-01-06,QQQ,305.53,310.39,304.52,306.07,53062950,306.7625,307.5884,298.0456,275.2608,...,53.0630,30.3734,8.0394,8.3704,8.9704,7.7704,-0.3310,0.3558,0.4228,-0.0276
2021-01-07,QQQ,308.80,314.33,308.77,313.48,30540630,312.5150,308.2042,298.6131,275.7543,...,30.5406,30.3893,8.9685,8.4164,9.0164,7.8164,0.5521,0.5674,0.4934,0.0460
2021-01-08,QQQ,315.82,317.86,313.58,317.51,34118733,316.6150,309.2556,299.3190,276.2955,...,34.1187,30.7445,9.6123,8.5084,9.1084,7.9084,1.1039,0.7060,0.5412,0.0920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-31,QQQ,367.24,367.60,361.68,362.54,67011405,363.5900,354.3468,353.6093,360.6706,...,67.0114,71.1906,0.8059,-1.0645,-0.4645,-1.6645,1.8704,0.4074,0.0392,0.1559
2022-04-01,QQQ,362.81,363.61,358.59,361.85,54527761,361.4750,355.2379,353.9177,360.6812,...,54.5278,69.6036,0.5145,-0.9430,-0.3430,-1.5430,1.4575,0.3085,0.0107,0.1215
2022-04-04,QQQ,362.71,369.31,362.44,369.30,45950171,367.5875,356.7816,354.4538,360.7727,...,45.9502,67.3509,1.3295,-0.7682,-0.1682,-1.3682,2.0977,0.5361,0.0915,0.1748


In [47]:
x = gen_chart(ticker, df, image_path="c:\\temp")

c:\temp/QQQ-00.png


In [48]:
x

{'ticker': 'QQQ', 'file_img': 'c:\\temp/QQQ-00.png'}

In [87]:
_query_tickers()

['QQQ']

In [65]:
tickers_ = df1['ticker'].to_list()
tickers_str = str(tickers_).replace('[','(').replace(']',')')
date_range = ('20210101','20220601')

sql_stmt = f'''
SELECT * 
FROM {table_name} 
where ticker in {tickers_str} 
    and date_ between '{date_range[0]}' and '{date_range[1]}'
order by ticker, date_
;
'''
print(sql_stmt)


SELECT * 
FROM quote_ta 
where ticker in ('QQQ') 
    and date_ between '20210101' and '20220601'
order by ticker, date_
;



In [38]:
# c.execute(f'SELECT * FROM {table_name} order by ticker,date_ limit 10;')
cur.execute(f'SELECT * FROM {table_name} limit 10;')
rows = cur.fetchall()

In [39]:
rows

[('QQQ',
  '19990310',
  44.334,
  44.361,
  43.605,
  44.279,
  12066145,
  44.131,
  44.131,
  44.131,
  44.131,
  44.6602,
  43.6018,
  44.887,
  43.375,
  12.0661,
  12.0661,
  None,
  None,
  None,
  None,
  None),
 ('QQQ',
  '19990311',
  44.601,
  44.856,
  43.624,
  44.488,
  22347406,
  44.364,
  44.2553,
  44.2498,
  44.2483,
  44.9749,
  43.5357,
  45.2534,
  43.2463,
  22.3474,
  17.4638,
  None,
  None,
  None,
  None,
  None),
 ('QQQ',
  '19990312',
  44.333,
  44.36,
  43.061,
  43.414,
  20164977,
  43.5622,
  43.9928,
  44.0114,
  44.0165,
  44.7945,
  43.1912,
  45.1214,
  42.9014,
  20.165,
  18.4557,
  None,
  None,
  None,
  None,
  None),
 ('QQQ',
  '19990315',
  43.734,
  44.707,
  43.272,
  44.658,
  14690628,
  44.3237,
  44.0928,
  44.0942,
  44.0949,
  44.9686,
  43.217,
  45.2955,
  42.893,
  14.6906,
  17.3688,
  None,
  None,
  None,
  None,
  None),
 ('QQQ',
  '19990316',
  44.849,
  45.227,
  44.36,
  45.041,
  11313996,
  44.9172,
  44.3044,
  44.2723,


In [76]:
from datetime import datetime,timedelta

In [77]:
to_date = datetime.now()

In [78]:
datetime.strftime(to_date, '%Y%m%d')

'20220409'

In [81]:
from_date = to_date + timedelta(days=-400)

In [82]:
datetime.strftime(from_date, '%Y%m%d')

'20210305'

In [34]:
x = 12.111111111111
round(x, 4)

12.1111

In [92]:
cols = """
ticker      dates   opens   highs    lows  closes   volumes      w_p  
ema_fast    ema_slow    ema_fast_d  ema_slow_u  ema_slow_d 
vol    vol_avg       rsi   rsi_avg     rsi_u     rsi_d  rsi_signal 
"""

In [103]:
columns = []
for c in [c.strip().split() for  c in cols.split("\n") if c.strip()]:
    for j in c:
        columns.append(j)

In [99]:
len(columns), ",".join(columns)

(20,
 'ticker,dates,opens,highs,lows,closes,volumes,w_p,ema_fast,ema_slow,ema_fast_d,ema_slow_u,ema_slow_d,vol,vol_avg,rsi,rsi_avg,rsi_u,rsi_d,rsi_signal')

In [101]:
vals = """
QQQ 2022-04-06  355.65  356.78  350.56  353.24  89720554  353.455   
357.009524  354.724103    352.329906  362.337633  347.110573   
89.720554  69.083925 -0.583806 -0.653224 -0.053224 -1.253224    0.069417 
"""

In [102]:
values = []
for c in [c.strip().split() for  c in vals.split("\n") if c.strip()]:
    for j in c:
        values.append(j)
len(values), ",".join(values)

(20,
 'QQQ,2022-04-06,355.65,356.78,350.56,353.24,89720554,353.455,357.009524,354.724103,352.329906,362.337633,347.110573,89.720554,69.083925,-0.583806,-0.653224,-0.053224,-1.253224,0.069417')